# Regression of Socioeconomic Factors of COVID-19 Outcomes

This work is an extension of a group project started in UC Berkeley's MIDS program. That project was a group effort from Jade Chia-Chun, Sirak Ghebremusse, Don Moon, and myself. That project focused on visualization and is available elsehwere in this repository. This notebook aims to use regression to ascertain which socioeconomic factors are most strongly realted to COVID-19 prevalence and lethality. This is <b>NOT</b> a declaration of any causal relationship, it is merely an excercise conducted to observe interesting correlations. In order to ascribe causality to any of these covariates one would need to look for natural experiments occurring in the world and use that information to determine a causal result.

### Imports

In [418]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
import altair as alt

### Data Cleaning

The dataset I will import below was created from a variety of sources (listed below). The notebook used to create it is available elsewhere in this repository.

This dataset was created for the purpose of creating visualizations in Tableau (the links to which are also available elsewhere in this repository). Next I will reduce the size of the data to what is relevant for modeling.

##### Data Sources

https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/

https://www.ers.usda.gov/data-products/county-level-data-sets/

https://github.com/CSSEGISandData/COVID-19/blob/master/README.md

https://covidtracking.com/race

https://www.nber.org/research/data/us-intercensal-county-population-data-age-sex-race-and-hispanic-origin


In [433]:
data = pd.read_csv("FINAL_w209_dataset_v3.csv")

In [434]:
fips_dict = {}

for i in range(data.shape[0]):
    fips_dict[data.countyFIPS.iloc[i]] = data['County Name'].iloc[i] +", "+ data['State'].iloc[i]
    
#fips_dict[1001]

For the purpose of visualization, I created many columns in this dataset. We do not need all of them for this segment of the analysis. The ones I will keep for modeling purposes are:
* countyFIPS
* 11/30/20
* 11/30/20_deaths
* PCTPOVALL_2018
* Civilian_labor_force_2018
    * will be converted to a percentage of state population
* Unemployment_rate_2018
* Median_Household_Income_2018
* population
* pct_nonwhite
    * this variable does not currently exist but will be created using tot_pop, wa_male, and wa_female

In [435]:
#Cleaning data by removing columns I won't use and reformatting columns that need it.

data = data[["countyFIPS",
             "County Name",
             "State",
             "11/30/20",
             "11/30/20_deaths",
             "PCTPOVALL_2018",
             "Civilian_labor_force_2018",
             "Unemployment_rate_2018",
             "Median_Household_Income_2018",
             "tot_pop",
             "wa_male",
             "wa_female",
             "population",]]

data['pct_nonwhite'] = 1 - (data['wa_male'] + data['wa_female']) /data['tot_pop']

data = data.replace(",","", regex = True)

data['Civilian_labor_force_2018'] = data['Civilian_labor_force_2018'].astype(float)
data['Median_Household_Income_2018'] = data['Median_Household_Income_2018'].astype(float)

data['pct_civ_labor_force'] = data['Civilian_labor_force_2018']/data['population']


data=data.dropna()
data = data[["countyFIPS",
             "County Name",
             "State",
             "11/30/20",
             "11/30/20_deaths",
             "PCTPOVALL_2018",
             "pct_civ_labor_force",
             "Unemployment_rate_2018",
             "Median_Household_Income_2018",
             "population",
             "pct_nonwhite"]]

In [436]:
#Checking that all columns are numeric. They are.

for col in data.columns:
    print(data[col].dtypes)

int64
object
object
int64
int64
float64
float64
float64
float64
float64
float64


### EDA

In [437]:
cols = ["PCTPOVALL_2018",
      "pct_civ_labor_force",
      "Unemployment_rate_2018",
      "Median_Household_Income_2018",
      "population",
      "pct_nonwhite"]
for col in cols:
    print(col, ":\n ", np.corrcoef(data["11/30/20"], data[col])[1][0], "\n ", np.corrcoef(data["11/30/20_deaths"], data[col])[1][0])

PCTPOVALL_2018 :
  -0.07351838006313716 
  -0.03920415314605741
pct_civ_labor_force :
  0.08764613601260306 
  0.05889570232694215
Unemployment_rate_2018 :
  -0.02708487381571131 
  0.008265953832854462
Median_Household_Income_2018 :
  0.21482860543142582 
  0.18674564398095006
population :
  0.9513052193575109 
  0.8253169437130466
pct_nonwhite :
  0.1423774614479918 
  0.16602907438781406


We see relatively high correlations between cases of COVID-19 and Median_Household_Income. There is also an appreciable correlation between COVID-19 cases and pct_nonwhite. These trends are far overshadowed by population of course, but that makes sense as to have high cases a high population is a prerequisite. This will have to be cancelled out by looking at cases/population, to understand the prevalence of COVID-19 relative to the size of the county.

In [438]:
data['case_rate'] = data['11/30/20']/data['population']
data['death_rate'] = data['11/30/20_deaths']/data['11/30/20']
data.head()

,countyFIPS,County Name,State,11/30/20,11/30/20_deaths,PCTPOVALL_2018,pct_civ_labor_force,Unemployment_rate_2018,Median_Household_Income_2018,population,pct_nonwhite,case_rate,death_rate
1,1001,Autauga County,AL,2780,42,13.8,0.468883,3.6,59338.0,55869.0,0.214011,0.049759,0.015108
2,1003,Baldwin County,AL,8890,98,9.8,0.426606,3.6,57588.0,223234.0,0.125647,0.039824,0.011024
3,1005,Barbour County,AL,1178,11,30.9,0.340841,5.1,34382.0,24686.0,0.495784,0.047719,0.009338
4,1007,Bibb County,AL,1196,17,21.8,0.384255,3.9,46064.0,22394.0,0.233602,0.053407,0.014214
5,1009,Blount County,AL,2997,40,13.2,0.433525,3.5,50412.0,57826.0,0.037032,0.051828,0.013347


In [439]:
for col in cols:
    print(col, ":\n  ", np.corrcoef(data["case_rate"], data[col])[1][0],"\n  ", np.corrcoef(data["death_rate"], data[col])[1][0])

PCTPOVALL_2018 :
   0.12670773886350234 
   0.20991468725906093
pct_civ_labor_force :
   0.036468703807629205 
   -0.1836192132369909
Unemployment_rate_2018 :
   -0.13709677129642528 
   0.1819217131627579
Median_Household_Income_2018 :
   -0.1385139402532819 
   -0.11667974257597762
population :
   -0.07171463803619074 
   0.060627037549056656
pct_nonwhite :
   0.09120690711105092 
   0.2707978262651606


Now that the population issue has been rectified by looking at rate metrics for COVID-19 prevalence and lethality we see the correlation drop to a more reasonable level. Now the most relevant correlations seem to be with poverty rate, which is positively associated with COVID-19 cases and deaths, unemployment rate, which is negatively associated with cases and positively associated with deaths, and nonwhite population, which is weakly positively correlated with cases but more convincingly correlated with deaths.

In [445]:
x_var = 'pct_nonwhite'
#x_var = 'PCTPOVALL_2018'
y_var = 'case_rate'
#y_var = 'death_rate'

title_text = x_var ,"vs", y_var

alt.Chart(data).mark_circle(opacity = 0.3, color = 'green').encode(
    x = x_var,
    y = y_var,
    tooltip = ['County Name', 'State']
).properties(
    title=title_text,
    width = 600,
    height = 400
)

alt.Chart(...)

### Separate out Train and Test Data

In [446]:
X = data[["PCTPOVALL_2018",
      "pct_civ_labor_force",
      "Unemployment_rate_2018",
      "Median_Household_Income_2018",
      "population",
      "pct_nonwhite"]]
#X.columns = ['pct_pov_18', 'pct_civ_labor_force', 'unemployment_rate_18', 'median_hh_inc_18', 'population', 'pct_nonwhite']
Y_cases = pd.DataFrame(data["case_rate"])
Y_deaths = pd.DataFrame(data["death_rate"])

In [447]:
X.shape, Y_cases.shape, Y_deaths.shape

((3108, 6), (3108, 1), (3108, 1))

In [448]:
X_train_c, X_test_c, Y_train_c, Y_test_c = train_test_split(X, Y_cases, test_size = 0.2)
X_train_d, X_test_d, Y_train_d, Y_test_d = train_test_split(X, Y_deaths, test_size = 0.2)

scaler = MinMaxScaler()
X_train_c = scaler.fit_transform(X_train_c)
X_test_c = scaler.transform(X_test_c)
Y_train_c = scaler.fit_transform(Y_train_c)
Y_test_c = scaler.transform(Y_test_c)


X_train_d = scaler.fit_transform(X_train_d)
X_test_d = scaler.transform(X_test_d)
Y_train_d = scaler.fit_transform(Y_train_d)
Y_test_d = scaler.transform(Y_test_d)

In [449]:
X_train_c.shape, X_test_c.shape, Y_train_c.shape, Y_test_c.shape, X_train_d.shape, X_test_d.shape, Y_train_d.shape, Y_test_d.shape

((2486, 6),
 (622, 6),
 (2486, 1),
 (622, 1),
 (2486, 6),
 (622, 6),
 (2486, 1),
 (622, 1))

First I ran some univariate regressions to see if those have any power to explain COVID-19 case rate or death rate. I did not find any of the 6 variables chosen above to be capable of explaining much of the variation on their own. It does seem that they have some use in explaining COVID-19 death rates.

In [460]:
cols = range(6)
for col in cols:
    reg_c = LinearRegression().fit(pd.DataFrame(X_train_c[:, col]), Y_train_c)
    reg_d = LinearRegression().fit(pd.DataFrame(X_train_d[:, col]), Y_train_d)
    print(col, round(reg_c.score(pd.DataFrame(X_test_c[:, col]), Y_test_c), 5), round(reg_d.score(pd.DataFrame(X_test_d[:, col]), Y_test_d),5))

0 -0.00543 0.01999
1 -0.00689 0.03617
2 -0.01049 0.03653
3 0.0044 -0.00739
4 -0.01051 0.00144
5 -0.00085 0.06597


Next, I looked at bivariate regression. Below are the results of COVID-19 cases regressed on all two-variable combinations. Again, explanatory power is very low, albeit slightly better for deaths.

In [462]:
cols = range(6)
combos = range(5)
for col in cols:
    for combo in combos:
        if col>combo:
            reg_c = LinearRegression().fit(pd.DataFrame(X_train_c[:, col]), Y_train_c)
            reg_d = LinearRegression().fit(pd.DataFrame(X_train_d[:, col]), Y_train_d)
            print(col, combo, round(reg_c.score(pd.DataFrame(X_test_c[:, col]), Y_test_c), 5), round(reg_d.score(pd.DataFrame(X_test_d[:, col]), Y_test_d), 5))
        
        


1 0 -0.00689 0.03617
2 0 -0.01049 0.03653
2 1 -0.01049 0.03653
3 0 0.0044 -0.00739
3 1 0.0044 -0.00739
3 2 0.0044 -0.00739
4 0 -0.01051 0.00144
4 1 -0.01051 0.00144
4 2 -0.01051 0.00144
4 3 -0.01051 0.00144
5 0 -0.00085 0.06597
5 1 -0.00085 0.06597
5 2 -0.00085 0.06597
5 3 -0.00085 0.06597
5 4 -0.00085 0.06597


Finally, I run a regression that include all 6 of the variables shown above. Here we see some amount of explanatory power, at least more than is shown in univariate and bivariate analysis. We are able to explain about 5% of the difference in case rate between counties and 7% of the difference in death rate using the above metrics (poverty rate, percent civilian labor force, unemployment rate, median household income, population, and percent nonwhite population).

In [484]:
col_dict = {0: "Poverty Rate (2018)     ",
            1: "Pct Civilian Labor Force",
            2: "Unemployment Rate (2018)",
            3: "Median HH Income (2018) ",
            4: "Population              ",
            5: "Pct Nonwhite            "
}

In [475]:
reg_cases = LinearRegression().fit(X_train_c, Y_train_c)
print("Cases:", round(reg_cases.score(X_test_c, Y_test_c),5))

reg_deaths = LinearRegression().fit(X_train_d, Y_train_d)
print("Deaths:", round(reg_deaths.score(X_test_d, Y_test_d),5))

Cases: 0.04845
Deaths: 0.0732


In [485]:
importance = reg_cases.coef_[0]
for i,v in enumerate(importance):
    print(col_dict[i],"      ",round(v, 4))

Poverty Rate (2018)             0.1811
Pct Civilian Labor Force        0.1538
Unemployment Rate (2018)        -0.4515
Median HH Income (2018)         -0.1259
Population                      -0.1832
Pct Nonwhite                    0.0682


In [486]:
importance = reg_deaths.coef_[0]
for i,v in enumerate(importance):
    print(col_dict[i],"      ",round(v, 4))

Poverty Rate (2018)             0.0165
Pct Civilian Labor Force        -0.0738
Unemployment Rate (2018)        0.0484
Median HH Income (2018)         -0.026
Population                      0.117
Pct Nonwhite                    0.0969


Looking at feature importance, we see that nonwhite population is the most useful of these covariates in explaining differences in county level COVID-19 case and mortality rates. It seems that counties with higher nonwhite populations have been hit harder by COVID-19. I took a deeper look into this using Tableau <a href = "https://public.tableau.com/profile/henry.bazakas#!/vizhome/w209_racial_disproportionate/DisproportionateImpactDash">here</a>.